In [1]:

# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder
from sklearn.utils.class_weight import compute_class_weight
import catboost as cat
from catboost import Pool
import lightgbm as lgb
from lightgbm import plot_tree as lgb_plot_tree
import xgboost as xgb
from xgboost import plot_tree as xgb_plot_tree
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold, cross_val_score
from sklearn.metrics import roc_auc_score
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s4e10/sample_submission.csv
/kaggle/input/playground-series-s4e10/train.csv
/kaggle/input/playground-series-s4e10/test.csv
/kaggle/input/loan-approval-prediction/credit_risk_dataset.csv


In [2]:
seed = 42

In [3]:
original = pd.read_csv('/kaggle/input/loan-approval-prediction/credit_risk_dataset.csv')
original.dropna(subset=['person_emp_length', 'loan_int_rate'], inplace=True)

In [4]:
train = pd.read_csv('/kaggle/input/playground-series-s4e10/train.csv')
test = pd.read_csv('/kaggle/input/playground-series-s4e10/test.csv')

train.drop(columns='id', inplace=True)

In [5]:
# train['loan_to_income'] = ((train['loan_amnt'] / train['person_income']) - train['loan_percent_income']).astype('string').astype('category')
# train['age_income_interaction'] = (train['person_age'] * train['person_income']).astype('string').astype('category')
# train['loan_to_emp_length_ratio'] = (train['loan_amnt'] / train['person_emp_length'].replace({'None': original['person_emp_length'].mean()}).astype('float')).astype('string').astype('category')
# monthly_income = train['person_income'] / 12
# train['monthly_debt'] = (train['loan_amnt'] * (1 + train['loan_int_rate'].replace({'None': original['loan_int_rate'].mean()})) / 12)
# train['dti_ratio'] = (train['monthly_debt'] / monthly_income).astype('string').astype('category')
# train['monthly_debt'] = train['monthly_debt'].astype('string').astype('category')
# train['risk_flag'] = (np.where((train['cb_person_default_on_file'] == 'Y') & (train['loan_grade'].isin(['C', 'D', 'E'])), 1, 0))
# train['risk_flag'] = train['risk_flag'].astype('category')
# train['person_home_ownership'] = train['person_home_ownership'].astype('category')
# train['loan_intent'] = train['loan_intent'].astype('category')
# train['loan_grade'] = train['loan_grade'].astype('category')
# train['cb_person_default_on_file'] = train['cb_person_default_on_file'].astype('category')
# train['person_emp_length'] = train['person_emp_length'].astype('string').astype('category')
# train['loan_int_rate'] = (train['loan_int_rate'] * 100).astype('string').astype('category')
# train['loan_percent_income'] = (train['loan_percent_income'] * 100).astype('string').astype('category')

In [6]:
combined_df = pd.concat([train, original], axis=0)

In [7]:
combined_df = combined_df.reset_index(drop=True)
# combined_df['id'] = np.arange(0, combined_df.shape[0])

In [8]:
# def new_features(comined_df):
#     combined_df['loan_to_income'] = ((combined_df['loan_amnt'] / combined_df['person_income']) - combined_df['loan_percent_income']).astype('string').astype('category')
#     combined_df['age_income_interaction'] = (combined_df['person_age'] * combined_df['person_income']).astype('string').astype('category')
#     combined_df['loan_to_emp_length_ratio'] = (combined_df['loan_amnt'] / combined_df['person_emp_length'].replace({'None': original['person_emp_length'].mean()}).astype('float')).astype('string').astype('category')
#     monthly_income = combined_df['person_income'] / 12
#     combined_df['monthly_debt'] = (combined_df['loan_amnt'] * (1 + combined_df['loan_int_rate'].replace({'None': original['loan_int_rate'].mean()})) / 12)
#     combined_df['dti_ratio'] = (combined_df['monthly_debt'] / monthly_income).astype('string').astype('category')
#     combined_df['monthly_debt'] = combined_df['monthly_debt'].astype('string').astype('category')
#     combined_df['risk_flag'] = (np.where((combined_df['cb_person_default_on_file'] == 'Y') & (combined_df['loan_grade'].isin(['C', 'D', 'E'])), 1, 0))
#     combined_df['risk_flag'] = combined_df['risk_flag'].astype('category')
#     combined_df['person_home_ownership'] = combined_df['person_home_ownership'].astype('category')
#     combined_df['loan_intent'] = combined_df['loan_intent'].astype('category')
#     combined_df['loan_grade'] = combined_df['loan_grade'].astype('category')
#     combined_df['cb_person_default_on_file'] = combined_df['cb_person_default_on_file'].astype('category')
#     combined_df['person_emp_length'] = combined_df['person_emp_length'].astype('string').astype('category')
#     combined_df['loan_int_rate'] = (combined_df['loan_int_rate'] * 100).astype('string').astype('category')
#     combined_df['loan_percent_income'] = (combined_df['loan_percent_income'] * 100).astype('string').astype('category')
#     return combined_df

# def new_features(df):
#     # Calculate mean values for columns with 'None' values
#     emp_length_mean = df['person_emp_length'].replace('None', np.nan).astype(float).mean()
#     int_rate_mean = df['loan_int_rate'].replace('None', np.nan).astype(float).mean()

#     # Create new features
#     df['loan_to_income'] = ((df['loan_amnt'].astype(float) / df['person_income'].astype(float)) - df['loan_percent_income'].astype(float)).astype(str).astype('category')
#     df['age_income_interaction'] = (df['person_age'].astype(float) * df['person_income'].astype(float)).astype(str).astype('category')
#     df['loan_to_emp_length_ratio'] = (df['loan_amnt'].astype(float) / df['person_emp_length'].replace({'None': emp_length_mean}).astype(float)).astype(str).astype('category')

#     monthly_income = df['person_income'].astype(float) / 12
#     df['monthly_debt'] = (df['loan_amnt'].astype(float) * (1 + df['loan_int_rate'].replace({'None': int_rate_mean}).astype(float)) / 12)
#     df['dti_ratio'] = (df['monthly_debt'] / monthly_income).astype(str).astype('category')
#     df['monthly_debt'] = df['monthly_debt'].astype(str).astype('category')

#     df['risk_flag'] = (np.where((df['cb_person_default_on_file'] == 'Y') & (df['loan_grade'].isin(['C', 'D', 'E'])), 1, 0))
#     df['risk_flag'] = df['risk_flag'].astype('category')

#     df['person_home_ownership'] = df['person_home_ownership'].astype('category')
#     df['loan_intent'] = df['loan_intent'].astype('category')
#     df['loan_grade'] = df['loan_grade'].astype('category')
#     df['cb_person_default_on_file'] = df['cb_person_default_on_file'].astype('category')
#     df['person_emp_length'] = df['person_emp_length'].astype(str).astype('category')
#     df['loan_int_rate'] = (df['loan_int_rate'].astype(float) * 100).astype(str).astype('category')
#     df['loan_percent_income'] = (df['loan_percent_income'].astype(float) * 100).astype(str).astype('category')

#     return df

In [9]:
def new_features(df):
    # Calculate mean values for columns with 'None' values
    emp_length_mean = df['person_emp_length'].replace('None', np.nan).astype(float).mean()
    int_rate_mean = df['loan_int_rate'].replace('None', np.nan).astype(float).mean()

    # Create new features
    df['loan_to_income'] = ((df['loan_amnt'].astype(float) / df['person_income'].astype(float)) - df['loan_percent_income']).astype(float)
    df['age_income_interaction'] = (df['person_age'].astype(float) * df['person_income'].astype(float))
    df['loan_to_emp_length_ratio'] = (df['loan_amnt'].astype(float) / df['person_emp_length'].replace({'None': emp_length_mean}).astype(float))

    monthly_income = df['person_income'].astype(float) / 12
    df['monthly_debt'] = (df['loan_amnt'].astype(float) * (1 + df['loan_int_rate'].replace({'None': int_rate_mean}).astype(float)) / 12)
    df['dti_ratio'] = (df['monthly_debt'] / monthly_income)
    
    df['risk_flag'] = np.where((df['cb_person_default_on_file'] == 'Y') & (df['loan_grade'].isin(['C', 'D', 'E'])), 1, 0)
    df['risk_flag'] = df['risk_flag'].astype('category')

    # New features implementation
    df['income_to_age_ratio'] = df['person_income'].astype(float) / df['person_age'].astype(float)
    
    df['income_stability_flag'] = np.where((df['person_home_ownership'] != 'RENT') & (df['person_emp_length'].astype(float) > 5), 1, 0)
    df['income_stability_flag'] = df['income_stability_flag'].astype('category')

    df['interest_to_loan_ratio'] = df['loan_int_rate'].astype(float) / df['loan_amnt'].astype(float)

    df['employment_stability'] = np.where(df['person_emp_length'].astype(float) > 5, 'Stable', 'Unstable')
    df['employment_stability'] = df['employment_stability'].astype('category')

    df['loan_grade_risk_flag'] = np.where((df['loan_grade'].isin(['C', 'D', 'E'])) & (df['cb_person_default_on_file'] == 'Y'), 1, 0)
    df['loan_grade_risk_flag'] = df['loan_grade_risk_flag'].astype('category')

    df['debt_burden'] = df['monthly_debt'] / monthly_income

    df['short_term_loan_flag'] = np.where((df['person_emp_length'].astype(float) < 1) & (df['loan_amnt'].astype(float) > 10000), 1, 0)
    df['short_term_loan_flag'] = df['short_term_loan_flag'].astype('category')

    # Convert other columns to categorical
    df['person_home_ownership'] = df['person_home_ownership'].astype('category')
    df['loan_intent'] = df['loan_intent'].astype('category')
    df['loan_grade'] = df['loan_grade'].astype('category')
    df['cb_person_default_on_file'] = df['cb_person_default_on_file'].astype('category')
    df['person_emp_length'] = df['person_emp_length'].astype(str).astype('category')
    df['loan_int_rate'] = (df['loan_int_rate'].astype(float) * 100).astype(str).astype('category')
    df['loan_percent_income'] = (df['loan_percent_income'].astype(float) * 100).astype(str).astype('category')

    return df
new_features_list = [
    'loan_to_income',
    'age_income_interaction',
    'loan_to_emp_length_ratio',
    'monthly_debt',
    'dti_ratio',
    'risk_flag'
]
converted_to_categorical = [
    'person_home_ownership',
    'loan_intent',
    'loan_grade',
    'cb_person_default_on_file',
    'person_emp_length',
    'loan_int_rate',
    'loan_percent_income'
]

In [10]:
combined_df = new_features(combined_df)
test = new_features(test)

In [11]:
cols_to_categorical = ['person_age', 'person_home_ownership', 'person_emp_length', 'loan_intent', 'loan_amnt', 'loan_int_rate',
'loan_percent_income', 'loan_grade', 'cb_person_default_on_file', 'cb_person_cred_hist_length', 'risk_flag']

In [12]:
test[cols_to_categorical] = test[cols_to_categorical].astype(str).astype('category')
combined_df[cols_to_categorical] = combined_df[cols_to_categorical].astype(str).astype('category')

In [13]:
combined_df.dtypes

person_age                    category
person_income                    int64
person_home_ownership         category
person_emp_length             category
loan_intent                   category
loan_grade                    category
loan_amnt                     category
loan_int_rate                 category
loan_percent_income           category
cb_person_default_on_file     category
cb_person_cred_hist_length    category
loan_status                      int64
loan_to_income                 float64
age_income_interaction         float64
loan_to_emp_length_ratio       float64
monthly_debt                   float64
dti_ratio                      float64
risk_flag                     category
income_to_age_ratio            float64
income_stability_flag         category
interest_to_loan_ratio         float64
employment_stability          category
loan_grade_risk_flag          category
debt_burden                    float64
short_term_loan_flag          category
dtype: object

In [14]:
test.dtypes

id                               int64
person_age                    category
person_income                    int64
person_home_ownership         category
person_emp_length             category
loan_intent                   category
loan_grade                    category
loan_amnt                     category
loan_int_rate                 category
loan_percent_income           category
cb_person_default_on_file     category
cb_person_cred_hist_length    category
loan_to_income                 float64
age_income_interaction         float64
loan_to_emp_length_ratio       float64
monthly_debt                   float64
dti_ratio                      float64
risk_flag                     category
income_to_age_ratio            float64
income_stability_flag         category
interest_to_loan_ratio         float64
employment_stability          category
loan_grade_risk_flag          category
debt_burden                    float64
short_term_loan_flag          category
dtype: object

In [15]:
# continuous_cols = [col for col in combined_df.columns if combined_df[col].nunique() > 50]
# categorical_cols = [col for col in combined_df.columns if col not in continuous_cols]

In [16]:
# for col in categorical_cols:
#         print('value_counts :', train[col].value_counts(), '\n')

In [17]:
status = train['loan_status'].value_counts()

In [18]:
# plt.pie(status, labels = status.index, autopct='%1.1f%%')
# plt.show()

In [19]:
X = combined_df.drop(columns='loan_status')
y = combined_df['loan_status']

cat_cols = X.select_dtypes(include='category').columns

In [20]:
# X.drop(columns='id', inplace=True)

In [21]:
#encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
# onehot = OneHotEncoder(sparse=False)
# onehot_cols = onehot.fit_transform(X[cat_cols])
#X[cat_cols] = encoder.fit_transform(X[cat_cols])

In [22]:
# test_onehot = onehot.transform(test[cat_cols])

In [23]:
# feature_names = onehot.get_feature_names_out(cat_cols)
# onehot_cols_df = pd.DataFrame(onehot_cols, columns=feature_names)
# onehot_test = pd.DataFrame(test_onehot, columns=feature_names)

In [24]:
# X.drop(columns=cat_cols, inplace=True)
# X = pd.concat([X, onehot_cols_df], axis=1)

In [25]:
# test.drop(columns=cat_cols, inplace=True)
# test = pd.concat([test, onehot_test], axis=1)

In [26]:
test.drop(columns='id', inplace=True)

In [27]:
test

,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_percent_income,cb_person_default_on_file,...,monthly_debt,dti_ratio,risk_flag,income_to_age_ratio,income_stability_flag,interest_to_loan_ratio,employment_stability,loan_grade_risk_flag,debt_burden,short_term_loan_flag
0,23,69000,RENT,3.0,HOMEIMPROVEMENT,F,25000,1576.0,36.0,N,...,34916.666667,6.072464,0,3000.000000,0,0.000630,Unstable,0,6.072464,0
1,26,96000,MORTGAGE,6.0,PERSONAL,C,10000,1268.0,10.0,Y,...,11400.000000,1.425000,1,3692.307692,1,0.001268,Stable,1,1.425000,0
2,26,30000,RENT,5.0,VENTURE,E,4000,1719.0000000000002,13.0,Y,...,6063.333333,2.425333,1,1153.846154,0,0.004298,Unstable,1,2.425333,0
3,33,50000,RENT,4.0,DEBTCONSOLIDATION,A,7000,890.0,14.000000000000002,N,...,5775.000000,1.386000,0,1515.151515,0,0.001271,Unstable,0,1.386000,0
4,26,102000,MORTGAGE,8.0,HOMEIMPROVEMENT,D,15000,1632.0,15.0,Y,...,21650.000000,2.547059,1,3923.076923,1,0.001088,Stable,1,2.547059,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39093,22,31200,MORTGAGE,2.0,DEBTCONSOLIDATION,B,3000,1037.0,10.0,N,...,2842.500000,1.093269,0,1418.181818,0,0.003457,Unstable,0,1.093269,0
39094,22,48000,MORTGAGE,6.0,EDUCATION,A,7000,603.0,15.0,N,...,4100.833333,1.025208,0,2181.818182,1,0.000861,Stable,0,1.025208,0
39095,51,60000,MORTGAGE,0.0,PERSONAL,A,15000,751.0,25.0,N,...,10637.500000,2.127500,0,1176.470588,0,0.000501,Unstable,0,2.127500,1
39096,22,36000,MORTGAGE,4.0,PERSONAL,D,14000,1562.0,39.0,Y,...,19390.000000,6.463333,1,1636.363636,0,0.001116,Unstable,1,6.463333,0


In [28]:
train

,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length,loan_status
0,37,35000,RENT,0.0,EDUCATION,B,6000,11.49,0.17,N,14,0
1,22,56000,OWN,6.0,MEDICAL,C,4000,13.35,0.07,N,2,0
2,29,28800,OWN,8.0,PERSONAL,A,6000,8.90,0.21,N,10,0
3,30,70000,RENT,14.0,VENTURE,B,12000,11.11,0.17,N,5,0
4,22,60000,RENT,2.0,MEDICAL,A,6000,6.92,0.10,N,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...
58640,34,120000,MORTGAGE,5.0,EDUCATION,D,25000,15.95,0.21,Y,10,0
58641,28,28800,RENT,0.0,MEDICAL,C,10000,12.73,0.35,N,8,1
58642,23,44000,RENT,7.0,EDUCATION,D,6800,16.00,0.15,N,2,1
58643,22,30000,RENT,2.0,EDUCATION,A,5000,8.90,0.17,N,3,0


In [29]:
# obj_cols= X.select_dtypes(include='object').columns
# X[obj_cols]= X[obj_cols].astype('category')

In [30]:
class_weights = compute_class_weight(class_weight='balanced', classes = np.unique(y), y=y)
class_weights

array([0.60004812, 2.9987975 ])

In [31]:
y_counts = y.value_counts()
scale_pos_weight = y_counts[0]/y_counts[1]
scale_pos_weight

4.997594997594998

In [32]:
cols = X.columns

In [33]:
training_data = pd.concat([X, y], axis=1)

In [34]:
# test[cols] = test[cols].astype(str).astype('category')
# training_data[cols] = training_data[cols].astype(str).astype('category')

In [35]:
training_data.dtypes

person_age                    category
person_income                    int64
person_home_ownership         category
person_emp_length             category
loan_intent                   category
loan_grade                    category
loan_amnt                     category
loan_int_rate                 category
loan_percent_income           category
cb_person_default_on_file     category
cb_person_cred_hist_length    category
loan_to_income                 float64
age_income_interaction         float64
loan_to_emp_length_ratio       float64
monthly_debt                   float64
dti_ratio                      float64
risk_flag                     category
income_to_age_ratio            float64
income_stability_flag         category
interest_to_loan_ratio         float64
employment_stability          category
loan_grade_risk_flag          category
debt_burden                    float64
short_term_loan_flag          category
loan_status                      int64
dtype: object

In [36]:
test.dtypes

person_age                    category
person_income                    int64
person_home_ownership         category
person_emp_length             category
loan_intent                   category
loan_grade                    category
loan_amnt                     category
loan_int_rate                 category
loan_percent_income           category
cb_person_default_on_file     category
cb_person_cred_hist_length    category
loan_to_income                 float64
age_income_interaction         float64
loan_to_emp_length_ratio       float64
monthly_debt                   float64
dti_ratio                      float64
risk_flag                     category
income_to_age_ratio            float64
income_stability_flag         category
interest_to_loan_ratio         float64
employment_stability          category
loan_grade_risk_flag          category
debt_burden                    float64
short_term_loan_flag          category
dtype: object

In [37]:
training_data.to_csv('training_prepped(encoding_final_boss).csv', index=False)
test.to_csv('test_prepped(encoding_final_boss).csv', index=False)

In [38]:
# models = {
# #     'cat': cat.CatBoostClassifier(cat_features=cat_cols, verbose=0),
# #     'xgb': xgb.XGBClassifier(enable_categorical=True, tree_method='hist', random_state=seed, scale_pos_weight=scale_pos_weight),
#      'lgbm': lgb.LGBMClassifier(verbose=0, random_state=seed, class_weight='balanced')
#  }

In [39]:
# X.columns

In [40]:
# X[feature_names] = X[feature_names].astype('category')

In [41]:
# def cv_score(models):
#     cv_scores = {}
#     for name, model in models.items():
#             cv_score = cross_val_score(model, X, y, scoring='roc_auc', n_jobs=-1)
#             print(f'{name} cv_scores :', cv_score, 'Mean :', cv_score.mean(), 'std :', cv_score.std())
            
#             model.fit(X, y)
            
#             if name =='xgb':
#                 plt.figure(figsize=(15,15))
#                 xgb_plot_tree(model, num_trees=1)
#                 plt.title('xgboost trees')
#                 plt.show()
                
#             if name == 'lgbm':
#                 plt.figure(figsize=(15,15))
#                 lgb_plot_tree(model, tree_index=1, figsize=(15,15))
#                 plt.title('lgbm trees')
#                 plt.show()
            
# cv_score(models)

In [42]:
# cat_cols = X.select_dtypes(include='category').columns
# print(cat_cols)
# models = {
#      'cat': cat.CatBoostClassifier(cat_features=cat_cols.to_list(), verbose=0, class_weights=class_weights),
#      'xgb': xgb.XGBClassifier(enable_categorical=True, scale_pos_weight=scale_pos_weight),
#      'lgbm': lgb.LGBMClassifier(verbose=0, categorical_feature=[f'name:{col}' for col in cat_cols.to_list()], class_weight='balanced')
#  }
# oof_results={}
# # test_idx = []
# def oof_preds(model_dict=models, n_splits=5):
#     test_ids = []
#     for name, model in models.items():
#         # new_df = pd.DataFrame()
#         preds_df = pd.DataFrame(columns=['index', 'loan_status'])
#         skf = StratifiedKFold(n_splits=n_splits, random_state=seed, shuffle=True)
#         for i, (train_index, test_index) in enumerate(skf.split(X, y)): 
#             # print('train_index :', len(train_index), 'test_index :', len(test_index))
#             # new_df = pd.concat([new_df,X.iloc[test_index]])
#             model.fit(X.iloc[train_index], y.iloc[train_index])
#             preds = model.predict_proba(X.iloc[test_index])[:,1]
#             if name=='lgbm':
#                 test_ids.extend(test_index)
#             fold_preds = pd.DataFrame({
#                 'loan_status': preds
#             })
#             preds_df = pd.concat([preds_df, fold_preds])
#             auc_score = roc_auc_score(y[test_index], preds)
#             # print(f'score for fold{i} :', auc_score)
#         oof_results[f'oof_{name}'] = preds_df['loan_status'].values
#     return test_ids, oof_results
# test_ids, oof_results = oof_preds()

In [43]:
# def oof_dict_to_df(oof_results):
#     df = pd.DataFrame()
#     for name, preds in oof_results.items():
#         df[name] = preds
#     return df
# combined_oof_df = oof_dict_to_df(oof_results)

In [44]:
# combined_oof_df.set_index(pd.Index(test_ids), inplace=True)

In [45]:
# combined_oof_df.sort_index(inplace=True)
# for cols in combined_oof_df.columns:
#     auc_score = roc_auc_score(y, combined_oof_df[cols].values)
#     print(f'Score of {cols} :', auc_score)

In [46]:
# trained_models = {}
# for name, model in models.items():
#     model.fit(X, y) 
#     trained_models[name] = model 

In [47]:
# lgbm_model = lgb.LGBMClassifier()
# lgbm_model.fit(combined_oof_df, y)

In [48]:
# def oof_preds_test(test, model_dict=trained_models):
#     df = pd.DataFrame()
#     for name, model in models.items():
#         preds = model.predict_proba(test)[:,1]
#         df[f'oof_{name}'] = preds
#     return df

In [49]:
# test_oof_df = oof_preds_test(test)
# test_oof_df.to_csv('tets_oof_df.csv', index=False)

In [50]:
# test_preds = lgbm_model.predict_proba(test_oof_df)[:,1]

In [51]:
# idx_df = pd.read_csv('/kaggle/input/playground-series-s4e10/test.csv')

In [52]:
# idx = idx_df['id']

In [53]:
# submission_df = pd.DataFrame({
#     'id':idx,
#     'loan_status':test_preds
# })
# submission_df.to_csv('submission.csv', index=False)

In [54]:
# submission_df

In [55]:
# combined_oof_df = pd.concat([combined_oof_df, y], axis=1)

In [56]:
# combined_oof_df.to_csv('combined_oof')